### CONFIGURATION

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [53]:
#VCF
#VCF_PATH = "/Users/peterpriestley/hmf/analyses/ensembleRuleTesting/"
VCF_PATH = "/Users/peterpriestley/hmf/analyses/161203_weirdKaryotype/"
# BED
#BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
#BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
#BED_FILE_NAME = "cosmicCancerCensus.bed" 
BED_FILE_NAME = "CPCT_Slicing.bed"
USE_BED = False
LOAD_FULL_FILE = True

# HEALTH CHECKS
HEALTH_CSV_PATH="/Users/peterpriestley/hmf/analyses/healthCheckAnalysis/"
HEALTH_CSV_FILENAME="healthCheckFile.csv"

In [5]:
# SPECIFY SAMPLES
VCF_SAMPLES=[]
for x in os.listdir(VCF_PATH):
    if x[-10:] == "sliced.vcf":
        VCF_SAMPLES.append(x)
VCF_SAMPLES
        #VCF_SAMPLES = ['160903_HMFregCPCT_FR10302737_FR10304566_CPCT02070012.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf']

['161123_HMFregCPCT_FR12244704_FR13275493_CPCT02060033.filtered_variants.vcf']

<h3> LOAD VCF

In [5]:
#Load all VCF_SAMPLES to DF
df = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    patientIDStart = VCF_SAMPLE.find("CPCT0")
    VCF_FILE_NAME = VCF_SAMPLE
    SAMPLE_NAMES = {VCF_SAMPLE[patientIDStart:patientIDStart+12]+'R':'normal',VCF_SAMPLE[patientIDStart:patientIDStart+12]+'T':'tumor'}
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[patientIDStart:patientIDStart+12],True,True,bed,LOAD_FULL_FILE)])
    else:
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[patientIDStart:patientIDStart+12],True)])

reading vcf file: 161123_HMFregCPCT_FR12244704_FR13275493_CPCT02060033.filtered_variants.vcf
reading VCF File line: 100000
reading VCF File line: 200000
reading VCF File line: 300000
reading VCF File line: 400000
reading VCF File line: 500000
reading VCF File line: 600000
reading VCF File line: 700000
reading VCF File line: 800000
reading VCF File line: 900000
reading VCF File line: 1000000
reading VCF File line: 1100000
reading VCF File line: 1200000
reading VCF File line: 1300000
reading VCF File line: 1400000
reading VCF File line: 1500000
reading VCF File line: 1600000
reading VCF File line: 1700000
reading VCF File line: 1800000
reading VCF File line: 1900000
reading VCF File line: 2000000
reading VCF File line: 2100000
reading VCF File line: 2200000
reading VCF File line: 2300000
reading VCF File line: 2400000
reading VCF File line: 2500000
reading VCF File line: 2600000
reading VCF File line: 2700000
reading VCF File line: 2800000
reading VCF File line: 2900000
reading VCF File 

In [6]:
#LOAD HEALTH_CHECKS
hc = pd.read_csv(HEALTH_CSV_PATH + HEALTH_CSV_FILENAME)
hc = hc[['SAMPLE','TUMOR_PERCENTAGE','TUMOR_TYPE','PIPELINE_VERSION']]
hc = hc[hc['SAMPLE'].str[12]=="T"]   # only tumor rows
hc['SAMPLE']=hc['SAMPLE'].str[:12]
hc = hc.set_index(['SAMPLE'])
df = pd.merge(df,hc,how='left', left_on='patientName',right_index=True)
df.fillna("Unknown", inplace=True)

<h3> High Level Stats

In [3]:
def loadPON(aPath,aPONFile):
    myPON = []
    with open(aPath + aPONFile, 'r') as f:
        for line in f:
            line = line.strip('\n')
            splitLine = line.split('\t')
            myPON.append(splitLine)
    dfPON = pd.DataFrame(myPON)
    dfPON.columns = ['chrom','pos','ref','alt','PONCount']
    return dfPON

In [4]:
dfPON = loadPON("/Users/peterpriestley/hmf/analyses/PON/779filePON/","PON.tsv")

In [54]:
bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)

In [26]:
def calculateInHMFBed(chromosome,position):
    for bedRow in bed:
        if chromosome<bedRow[0]:
            return False
        elif (chromosome==bedRow[0]) and (position>int(bedRow[1])) and (position<int(bedRow[2])):
            return True
    return False        
        
    #return dfCNV[(dfCNV.chrom==chromosome)&(dfCNV.posStart<position)&(dfCNV.posEnd>=position)]['copyNum'].mean()

In [59]:
dfPON[dfPON.inHMFBed==True]

,chrom,pos,ref,alt,PONCount,inHMFBed
153902,1,11199518,T,C,6,True
153903,1,11199541,G,A,6,True
154784,1,11298631,G,A,6,True
539892,1,43815035,C,T,14,True
1244732,1,115256529,T,A,2,True
1244733,1,115256529,T,C,11,True
1244734,1,115256530,G,T,8,True
1244743,1,115258745,C,G,2,True
2732706,2,16082031,C,T,6,True
2732707,2,16082077,G,T,2,True


In [56]:
dfPON['inHMFBed']=np.vectorize(calculateInHMFBed)( dfPON['chrom'],pd.to_numeric(dfPON.pos))

In [16]:
bed[1:10]

[['1', '115247090', '115259515', 'ENST00000369535.4 (NRAS)'],
 ['1', '156830686', '156851434', 'ENST00000524377.1 (NTRK1)'],
 ['2', '16080686', '16087129', 'ENST00000281043.3 (MYCN)'],
 ['2', '29415640', '30144432', 'ENST00000389048.3 (ALK)'],
 ['2', '47630108', '47710367', 'ENST00000233146.2 (MSH2)'],
 ['2', '48010221', '48037240', 'ENST00000234420.5 (MSH6)'],
 ['2', '58386382', '58468485', 'ENST00000402135.3 (FANCL)'],
 ['2', '79412357', '80875904', 'ENST00000466387.1 (CTNNA2)'],
 ['2', '209100989', '209119046', 'ENST00000415913.1 (IDH1)']]

In [18]:
dfPON.pos.max()

'99999995'

In [55]:
mysum = 0.0
sum([int(bedRow[2])-int(bedRow[1]) for bedRow in bed])

77449

In [58]:
print "Hello"

Hello
